<a href="https://colab.research.google.com/github/Muhammad-Usama-07/DeepLearning-Work/blob/main/HousePricePrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [2]:
from keras.datasets import boston_housing


# Loading Data

In [3]:
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

65536/57026 [==================================] - 0s 0us/step


# Shape of Data

In [4]:
print('train data shape:',train_data.shape)
print('train data shape:', test_data.shape)


train data shape: (404, 13)
train data shape: (102, 13)
